## 图像样本集处理

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import cv2 as cv
from keras.utils import np_utils, to_categorical
import os
import glob
from imutils import paths   #基于opencv的比较好用的工具箱
import imutils

from keras.callbacks import TensorBoard
from keras.utils import plot_model

from PIL import Image, ImageFile, ImageEnhance
ImageFile.LOAD_TRUNCATED_IMAGES = True

Using TensorFlow backend.


### 0. 文件路径加载

In [2]:
fileDir='F:\\ipython\\Computer_Vision\\GC\\trainingC\\' #图像样本文件路径
total_images = 0
major = os.listdir(fileDir)  
full_path = []
print("Loading file structure...")
for a in major:
      full_path.append(fileDir + a + "\\") 
print(full_path)

Loading file structure...
['F:\\ipython\\Computer_Vision\\GC\\trainingC\\C0paper_recycle\\', 'F:\\ipython\\Computer_Vision\\GC\\trainingC\\C1paper_gan\\', 'F:\\ipython\\Computer_Vision\\GC\\trainingC\\C2paperpacking\\', 'F:\\ipython\\Computer_Vision\\GC\\trainingC\\C3plastic\\', 'F:\\ipython\\Computer_Vision\\GC\\trainingC\\C4plastic_bottle\\', 'F:\\ipython\\Computer_Vision\\GC\\trainingC\\C5plastic_product\\', 'F:\\ipython\\Computer_Vision\\GC\\trainingC\\C6plastic_packing\\', 'F:\\ipython\\Computer_Vision\\GC\\trainingC\\C7metal\\', 'F:\\ipython\\Computer_Vision\\GC\\trainingC\\C8glass\\', 'F:\\ipython\\Computer_Vision\\GC\\trainingC\\C9tile\\', 'F:\\ipython\\Computer_Vision\\GC\\trainingC\\CAfabric\\', 'F:\\ipython\\Computer_Vision\\GC\\trainingC\\CBwood\\', 'F:\\ipython\\Computer_Vision\\GC\\trainingC\\CCdust\\', 'F:\\ipython\\Computer_Vision\\GC\\trainingC\\CDwet\\', 'F:\\ipython\\Computer_Vision\\GC\\trainingC\\CEfood_waste\\', 'F:\\ipython\\Computer_Vision\\GC\\trainingC\\CFfood

#### 检查当前目录下损坏的图片文件并删除

In [5]:
# just for unit test
if __name__ == '__main__':

    badFilesList = []
    for i in full_path:
        curDir = i
        for root, dirs, files in os.walk(curDir):
            # print(files)
            # 检查当前目录中的损坏的图片文件
            for each in files:
                # for each in os.listdir('./'):
                if each.endswith('.png') or each.endswith('.jpg') or each.endswith('.gif') or each.endswith(
                        '.JPG') or each.endswith('.PNG') or each.endswith('.GIF') or each.endswith(
                    '.jpeg') or each.endswith(
                    '.JPEG'):
                    # print(each)

                    try:

                        im = Image.open(os.path.join(root, each))
                        # im.show()
                    except Exception as e:
                        print('Bad file:', os.path.join(root, each))
                        badFilesList.append(os.path.join(root, each))

        # 删除损坏的文件
        if len(badFilesList) != 0:
            for each in badFilesList:
                try:
                    os.remove(each)
                except Exception as e:
                    print('Del file: %s failed, %s' % (each, e))

        pass

D:\Anaconda\lib\site-packages\PIL\TiffImagePlugin.py:780: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


### 1. 数据集处理方法一

### 1.1 数据增强 均衡数据集

In [3]:
#旋转
def rotation(root_path, img_name):
    img = Image.open(os.path.join(root_path, img_name))
    if img.mode == "P" or 'RGBA':
        img = img.convert('RGB')    
    rotation_img = img.rotate(20) #旋转角度
    # rotation_img.save(os.path.join(root_path,img_name.split('.')[0] + '_rotation.jpg'))
    return rotation_img

In [12]:
#翻转
def flip(root_path,img_name):   #翻转图像
    
    img = Image.open(os.path.join(root_path, img_name))
    if img.mode == "P" or 'RGBA':
        img = img.convert('RGB')    
    filp_img = img.transpose(Image.FLIP_LEFT_RIGHT)
    # filp_img.save(os.path.join(root_path,img_name.split('.')[0] + '_flip.jpg'))
    return filp_img

In [11]:
# 针对不同分类图像文件夹，选用数据增强方式执行，如尘土类（15）图片较少，可选用旋转方式增强样本，纸巾干垃圾图集较多，可不增强，实现样本均衡
# 旋转
for lable in range(15,16):
    #print(full_path[lable])
    imageDir = full_path[lable]
    saveDir = full_path[lable]
    i=0
    for name in os.listdir(imageDir):
        i=i+1
        saveName="xz"+str(i)+".jpg"
        saveImage=rotation(imageDir,name)
        saveImage.save(os.path.join(saveDir,saveName))

In [13]:
#翻转
for lable in range(1,2):
    #print(full_path[lable])
    imageDir = full_path[lable]
    saveDir = full_path[lable]
    i=0        
    for name in os.listdir(imageDir):
            i=i+1
            saveName="fz"+str(i)+".jpg"
            saveImage=flip(imageDir,name)
            saveImage.save(os.path.join(saveDir,saveName))

数据增加及随机采样，实现样本均衡

### 1.2 图像预处理 及 划分测试集训练集

In [14]:
RESOLUTION=192     ##分辨率大小设置
#将图片数字化       
print("Loading training images...")
X_train = []
for i in full_path:
    images_in_folder = 0
    label = i.split('\\')[1]    
    #label = label.split('-')[0]    一级分类
    labelIndex=all_labels.index(label)  ##读取标签索引
    print('label:',label,':',labelIndex)
    for file in glob.glob(i + "*.jpg"):
        try:
            img = cv.cvtColor(cv.imread(file),cv.COLOR_BGR2RGB)   ## 颜色空间转化
                                ##cv.imread(file)读取图像为数据  
                                ##cv.COLOR_BGR2RGB 转颜色通道
        except:
            Exception
            continue
        img = cv.resize(img, (RESOLUTION,RESOLUTION))  ##分辨率大小192*192统一尺寸  #由于电脑性能限制，没有提高分辨率训练，一般建议224及以上
        total_images+=1
        X_train.append(img)
        y_labels.append(labelIndex) ##同时读取标签
        images_in_folder += 1
    print("The total number of images in %s = %d" % (i,images_in_folder))
print("The total number of images in data = " + str(total_images))

Loading training images...
label: C0paper_recycle : 0
The total number of images in trainingC\C0paper_recycle\ = 1768
label: C1paper_gan : 1
The total number of images in trainingC\C1paper_gan\ = 2147
label: C2paperpacking : 2
The total number of images in trainingC\C2paperpacking\ = 1614
label: C3plastic : 3
The total number of images in trainingC\C3plastic\ = 2173
label: C4plastic_bottle : 4
The total number of images in trainingC\C4plastic_bottle\ = 1704
label: C5plastic_product : 5
The total number of images in trainingC\C5plastic_product\ = 2284
label: C6plastic_packing : 6
The total number of images in trainingC\C6plastic_packing\ = 2221
label: C7metal : 7
The total number of images in trainingC\C7metal\ = 1908
label: C8glass : 8
The total number of images in trainingC\C8glass\ = 2143
label: C9tile : 9
The total number of images in trainingC\C9tile\ = 2038
label: CAfabric : 10
The total number of images in trainingC\CAfabric\ = 2108
label: CBwood : 11
The total number of images i

In [15]:
 #查看有几类标签
print(np.unique(y_labels))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]


In [ ]:
#多分类编码转为onehot编码
y_labels = to_categorical(y_labels)  ##onehot编码可以结合1 文件的10，11块输出部分理解

In [16]:
# 正则化
#图片数据规范化，数据归一化，会减少损失函数的震荡，有助于减小损失函数提高精度。
X_train = np.array(X_train, np.float32) / 255.
# Normalization标准化处理、标准差
X_norm = (X_train - X_train.mean(axis=0))/X_train.std(axis = 0)
# 洗牌 打乱数据
from sklearn.utils import shuffle
X_train, y_labels = shuffle(X_norm, y_labels,random_state=0) 

In [17]:
# #保存处理好的数据与标签
# np.save('encoded/X_train.npy',X_train)
# np.save('encoded/y_labels.npy',y_labels)
# print("数据保存成功！")
# ## 运行成功后自动生成

数据保存成功！


#### 拆分训练集测试集

In [7]:
from sklearn.model_selection import train_test_split
Xtrain, Xvalid, Ytrain, Yvalid = train_test_split(X_train, y_labels, test_size=0.3, random_state=7)

In [8]:
#保存处理好的数据与标签
np.save('encoded/X_train_C_192.npy', Xtrain)
np.save('encoded/X_valid_C_192.npy', Xvalid)
np.save('encoded/Y_train_C_192.npy', Ytrain)
np.save('encoded/Y_valid_C_192.npy', Yvalid)
print("数据保存成功！")


数据保存成功！


本实验后续使用以上（方法一）处理后的数据集，进行自定义CNN实验及DenseNet迁移学习实验。

下文的方法二是另一种处理处理方式摘要

### 2. 数据集处理方法二

#### 2.1 划分训练集/测试集 图像文件

In [7]:
import random, shutil
def moveFile(fileDir):
        pathDir = os.listdir(fileDir)    #取图片的原始路径
        filenumber=len(pathDir)
        rate=0.3    #自定义抽取图片的比例
        picknumber=int(filenumber*rate) #按照rate比例从文件夹中取一定数量图片
        sample = random.sample(pathDir, picknumber)  #随机选取picknumber数量的样本图片
        #print (sample)
        for name in sample:
                shutil.move(fileDir+name, tarDir+name)  #将文件或目录(源)递归移动到另一个位置(目标)并返回目标。
        return

label_L = []
for i in full_path:
    label = i.split('\\')[5] #从路径中提取文件夹标签
    label_L.append(label)
    fileDir = i
    tarDir = 'F:\\ipython\\Computer_Vision\\GC\\trainingC\\validation\\'+label+'\\' # 目标（测试集）文件路径
    moveFile(fileDir)
    print('finishedfile:'+label)


1
finishedfile:C0paper_recycle
2
finishedfile:C1paper_gan
3
finishedfile:C2paperpacking
4
finishedfile:C3plastic
5
finishedfile:C4plastic_bottle
6
finishedfile:C5plastic_product
7
finishedfile:C6plastic_packing
8
finishedfile:C7metal
9
finishedfile:C8glass
10
finishedfile:C9tile
11
finishedfile:CAfabric
12
finishedfile:CBwood
13
finishedfile:CCdust
14
finishedfile:CDwet
15
finishedfile:CEfood_waste
16
finishedfile:CFfood_packaging


至此，已完成各子类别文件夹划分训练/测试集；由于子类别文件夹已经按照不同类别完成整理，因此直接按文件夹作为图片样本的标签，用来生成数据集。

后面部分将使用ImageDataGenerator.flow_from_directory(directory)方法来生成数据集，因此先构建训练/验证数据集的文件路径名：

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
 
import tensorflow as tf
 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
import os
import numpy as np
import matplotlib.pyplot as plt

In [3]:
train_dir = os.path.join(fileDir, 'train')
validation_dir = os.path.join(fileDir, 'validation')

# 各类样本分布统计
num_total_train, num_total_val=0,0
for i in range len(label_L):
    train_dir_i = os.path.join(train_dir, label_L[i])  
    validation_dir_i = os.path.join(validation_dir, label_L[i])  
    num_tr_i = len(os.listdir(train_dir_i))
    num_val_i = len(os.listdir(validation_dir_i))
    print(label_L[i]+' training images:\t',train_dir_i,'\t validation images:\t',validation_dir_i,'\n')
    num_total_train += num_tr_i
    num_total_val += num_val_i
print('Total training images:\t',num_total_train,'\t validation images:\t',num_total_val)

#### 2.2 构建数据集

In [ ]:
# flow_from_directory(directory): 以文件夹路径为参数,生成经过数据提升/归一化后的数据,在一个无限循环中无限产生batch数据
    
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150

train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

validation_image_generator = ImageDataGenerator(rescale=1./255)
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

之后直接以train_data_gen等作为数据集放进fit里训练即可